In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import nip

import os
import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

DEVICE = 'cuda:0'

In [2]:
#path_to_checkpoints = "/home/msst/repo/Quantization/logs/Llama-3.2-1B"
#chpnt_name = 'Llama-3.2-1B_lsq4bit128gs_init'

checkpoint_path = "/home/msst/repo/Quantization/logs/Llama-3.2-1B/Llama-3.2-1B_lsq3bit128gs_init"

qmodel = qlib.QuantizedLlamaForCausalLM.from_pretrained(
    checkpoint_path,
    torch_dtype="auto",
).to(DEVICE)

tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

LOL, Im quantized llama!


In [3]:
dataloader = qlib.QATDataset(
    config=nip.load('/home/msst/repo/Quantization/configs/data/wikitext_test_seqlen4096.yaml'),
    tokenizer=tokenizer
).get_dataloader()

with torch.amp.autocast('cuda', dtype=torch.bfloat16):
    res = qlib.evaluate(qmodel.half(), dataloader, print_times=25)
    print(res)


Token indices sequence length is longer than the specified maximum sequence length for this model (289077 > 131072). Running this sequence through the model will result in indexing errors
  4%|▍         | 3/70 [00:03<01:04,  1.04it/s]

163.4477298588659


  7%|▋         | 5/70 [00:04<00:49,  1.30it/s]

166.8670710269331


 10%|█         | 7/70 [00:05<00:43,  1.45it/s]

...

182.89555159966423


 24%|██▍       | 17/70 [00:11<00:31,  1.67it/s]

180.4595355248961


 26%|██▌       | 18/70 [00:11<00:34,  1.50it/s]


KeyboardInterrupt: 

In [4]:
# 9.07

In [5]:
# path_to_save = os.path.join(
# 	path_to_checkpoints, 
# 	f'{chpnt_name}_act4bit_calib2'
# )
# qmodel.half().save_pretrained(path_to_save)

In [6]:
### lukashevich icoherence + norm
# 39 ppl init
# 6.393 ptq
# 6.297 ptq + qat (CE, 150 steps 20 ga Adam)
# 6.227 ptq + qat (CE, 50 steps 50 ga Adam)

### MX acts
# 6.247 ptq + qat + fp8_e4m3 acts
# 6.287 ptq + qat + fp6_e3m2 acts
# 6.244 ptq + qat + fp6_e2m3 acts
# 6.498 ptq + qat + fp4_e2m1 acts

### INT acts
# 7.206 ptq + 8 bit acts calib
# 7.107 ptq + 8 bit acts ptq (7.040 short ptq first 6 blocks)
#  769  ptq + 4 bit acts calib
#  650  ptq + 4 bit acts calib2
#  ~450 ptq + 4 bit acts calib2 qat 20ga 10 steps


### 25.07 ppl init
## 7.016 ptq (ns=128, bs=2, ne=1)
## 7.545 ptq (ns=128, bs=2, ne=1, fp acts)

## 7.058 ptq (ns=128, bs=2, ne=2)
# 6.654 ptq + qat (KD+CE, 25 steps)
# 6.598 ptq + qat (CE, 25 steps)
# 6.547 ptq + qat (KD, 10 steps)
# 6.511 ptq + qat (KD, 5 steps)

## 6.797 ptq (ns=320, bs=5, ne=4)
## 6.779 ptq (ns=320, bs=5, ne=2)
# 6.579 ptq + qat (KD, 2 steps 1 ga Adam)
# 6.493 ptq + qat (KD, 5 steps 1 ga Adam) # PiQA 0.7584 # WINO 0.6606 # ArcE 0.7071 # ArcC 0.3712
# 7.372 ptq + qat (KD, 25 steps 1 ga Adam)
# 7.236 ptq + qat (KD, 10 steps 4 ga Adafactor)
# > 7.5 ptq + qat (CE, 10 steps 4 ga Adafactor)
# 6.567 ptq + qat (CE, 10 steps 4 ga Adam)
# 6.612 ptq + qat (CE, 10 steps 4 ga Adam small lr)
# 6.609 ptq + qat (CE, 25 steps 4 ga Adam)
# 7.449 ptq + qat (CE, 100 steps 2 ga Adam)
# 6.464 ptq + qat (CE, 20 steps 10 ga Adam)
# 6.390 ptq + qat (CE, 20 steps 20 ga Adam)
# 6.325 ptq + qat (CE, 125 steps 20 ga Adam)

### 39 ppl init
# 6.778 ptq (ns=128, bs=2, ne=2)
# 6.565 ptq + qat (CE, 10 steps 1 ga Adafactor)
# 6.478 ptq + qat (KD, 25 steps 1 ga Adafactor)
# 6.420 ptq + qat (KD+CE, 25 steps 1 ga Adafactor)

In [7]:
(0.383 + 0.710 + 0.761 + 0.665) / 4





0.62975